# Retrieve data from eNanomapper database
https://search.data.enanomapper.net/

- This notebook uses Apache Solr API and AMBIT REST API  (see Swagger interactive documentation at http://enanomapper.github.io/API/ )

In [ ]:
import pandas as pd
import os.path
import numpy as np
import datetime, time
import urllib3
import json
import sys
import ipywidgets as widgets

import logging
from logging.config import fileConfig
fileConfig('logging_endpoints_config.ini')

global logger
logger = logging.getLogger()

%run units.py

logger.debug('Started at %s \t%s',os.name, datetime.datetime.now())

urllib3.disable_warnings()

In [ ]:
%run solrapi.py

In [ ]:
%run enmapi.py

In [ ]:
http_pool=urllib3.PoolManager()

### Retrieve endpoints using AMBIT REST API

In [ ]:
ambit=AMBITQuery(key="study")
print(ambit.root)
data = ambit.get(http_pool)


In [ ]:
#Select endpoint
_sections={}

top_sections = []
for facet in data['facet']:
    #print("{}\t{}\t{}\t{}".format(facet['subcategory'],facet['endpoint'],facet['count'],facet['value']))
    top = facet['subcategory']
    if not top in top_sections:
        top_sections.append(top)
        
    if not top in _sections:
        _sections[top] = []
    
    _sections[top].append(facet['endpoint'])

top_widget = widgets.Dropdown(
    options=top_sections,
    value=top_sections[0],
    description='Select:',
    disabled=False,
)
display(top_widget)


In [ ]:
top=top_widget.value
category_widget = widgets.Dropdown(
    options=_sections[top],
    value=_sections[top][1],
    description='Endpoint:',
    disabled=False,
)
display(category_widget)

#### Setup the query

In [ ]:
freetext_widget=widgets.Text(
    value='NM220,NM101',
    description='Search',
    disabled=False
)
freetext_widget

In [ ]:
materialfilter=freetext_widget.value
if "" == materialfilter:
    materialfilter=None
    
section=category_widget.value
fields=None
if section in _fields:
    fields=_fields[section]

endpoint="*"
if section in _endpoint:
    endpoint=_endpoint[section]
    
settings = {'endpointfilter': ' effectendpoint_s: {}'.format(endpoint),
            'studyfilter': ' topcategory_s:({}) AND endpointcategory_s:({})'.format(top,section),            
            'query_guidance': None, 
            'query_organism': None,
            'fields' : fields
           }
    
logger.info(settings)  
logger.info("Free text query: {}".format(materialfilter))

### Run the query

In [ ]:

try:
    with open('./config.solr.json') as f:
        solr_services = json.load(f)
except:        
    solr_services = None
    logger.error("Something went wrong")

service_widget = widgets.Dropdown(
    options=solr_services.keys(),
    description='Service:',
    disabled=False,
)
display(service_widget)

name_widget=widgets.Text(
    placeholder='',
    description='User name:',
    disabled=False
)
password_widget = widgets.Password(description='Password:', placeholder='')
display(name_widget)
password_widget

In [ ]:
query=getSolrQuery(settings,url=solr_services[service_widget.value])
rows = []  

#logger.info(settings)
logger.info(query)
rows = sendSolrRequest(settings=settings,http=http_pool,textfilter=materialfilter,query=query,rows=rows,basic_auth="{}:{}".format(name_widget.value,password_widget.value))

In [39]:
#print("Substances: {}".format(len(rows)))
df = pd.DataFrame(rows)
df.to_csv(section+".nosmiles.txt",sep='\t',index=False)
#df.head()
df.head()

,public.name,r.purposeFlag,r.reliability,r.studyResultType,substance.name,substance.type,substance.uuid,value.endpoint,value.lo,value.qualifier.lo,...,x.params.T.Measurement temperature,x.params.T.Measurement viscosity,x.params.T.Number of runs,x.params.T.PDI,x.params.T.Refractive index of the medium,x.params.T.Refractive index of the sample,x.params.T.Scattering angle,x.params.Vial,x.params.guidance,x.reference_year
0,JRCNM01000a,,None,,NM-100 (TiO2 50-150 nm),NPO_1486,NNRG-18280a4a-45e9-adc0-df3b-125397b1255f,MEDIAN ECD,79.5,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20068/20078/20079/20069/20080,TEM,2016
1,JRCNM01000a,,None,,NM-100 (TiO2 50-150 nm),NPO_1486,NNRG-18280a4a-45e9-adc0-df3b-125397b1255f,MEAN ECD,86.3,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20068/20078/20079/20069/20080,TEM,2016
2,JRCNM01000a,,None,,NM-100 (TiO2 50-150 nm),NPO_1486,NNRG-18280a4a-45e9-adc0-df3b-125397b1255f,MODE ECD,70.2,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20068/20078/20079/20069/20080,TEM,2016
3,JRCNM01000a,,None,,NM-100 (TiO2 50-150 nm),NPO_1486,NNRG-18280a4a-45e9-adc0-df3b-125397b1255f,MEDIAN FERET MINIMAL DIAMETER,101.2,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20068/20078/20079/20069/20080,TEM,2016
4,JRCNM01000a,,None,,NM-100 (TiO2 50-150 nm),NPO_1486,NNRG-18280a4a-45e9-adc0-df3b-125397b1255f,MEAN FERET MINIMAL DIAMETER,110.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20068/20078/20079/20069/20080,TEM,2016


In [40]:
import numpy as np
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: red' if v else '' for v in is_max]

for criteria in ["value.lo"]:
    tmp = pd.pivot_table(df, values=criteria, index=['public.name'], columns=['x.oht.top','x.oht.section','x.guidance','value.endpoint','value.qualifier.lo','value.unit'], aggfunc=np.mean).reset_index()
    #tmp.style.highlight_null(null_color='red')
    
    display(tmp.style.apply(highlight_max,subset=top_sections))

In [41]:
tmp=df.groupby(by=["public.name","x.guidance","value.endpoint","value.qualifier.lo","value.unit"]).agg({"value.lo" : ["min","max","mean","std","count"]}).reset_index()
tmp.columns = ["_".join(x) for x in tmp.columns.ravel()]
print("Substances {}".format(tmp.shape[0]))
display(tmp)


Substances 27


,public.name_,x.guidance_,value.endpoint_,value.qualifier.lo_,value.unit_,value.lo_min,value.lo_max,value.lo_mean,value.lo_std,value.lo_count
0,JRCNM01000a,DLS,GLOBAL MEAN SIZE,mean,nm,288.100,288.10,288.100000,NaN,1
1,JRCNM01000a,DLS,GLOBAL MEAN SIZE,peak,nm,295.000,5210.00,1938.400000,2833.298904,3
2,JRCNM01000a,DLS,Z-AVE HYDRODYNAMIC DIAMETER,,nm,213.560,288.10,236.226000,31.154622,5
3,JRCNM01000a,NANOREG SUPPLIER,DIAMETER,>=,nm,50.000,50.00,50.000000,0.000000,2
4,JRCNM01000a,TEM,MEAN ECD,,nm,86.300,221.30,133.150000,42.435834,8
5,JRCNM01000a,TEM,MEAN FERET MINIMAL DIAMETER,,nm,94.000,204.00,135.200000,39.522977,8
6,JRCNM01000a,TEM,MEDIAN ECD,,nm,79.500,187.60,124.062500,32.329197,8
7,JRCNM01000a,TEM,MEDIAN FERET MINIMAL DIAMETER,,nm,94.000,165.10,123.687500,25.974627,8
8,JRCNM01000a,TEM,MODE ECD,,nm,70.200,186.60,117.950000,35.209374,8
9,JRCNM01000a,TEM,MODE FERET MINIMAL DIAMETER,,nm,88.900,136.00,114.712500,17.418991,8
